In [1]:
!pip uninstall folium -y
!pip install folium


Found existing installation: folium 0.16.0
Uninstalling folium-0.16.0:
  Successfully uninstalled folium-0.16.0
   ---------------------------------------- 0.0/108.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/108.4 kB ? eta -:--:--
   -------------- ------------------------ 41.0/108.4 kB 393.8 kB/s eta 0:00:01
   ----------------------------------- -- 102.4/108.4 kB 837.8 kB/s eta 0:00:01
   -------------------------------------- 108.4/108.4 kB 785.4 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import folium

In [3]:
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon


In [7]:
import requests
import pandas as pd
import io

# Define the URL for the CSV file
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Make a GET request to fetch the raw CSV data
response = requests.get(URL)

# Use io.BytesIO to create a buffer from the response content
spacex_csv_file = io.BytesIO(response.content)

# Read the CSV file into a DataFrame
spacex_df = pd.read_csv(spacex_csv_file)

# Display the first few rows of the DataFrame
print(spacex_df.head())


   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for index, launch_site in launch_sites_df.iterrows():
    lat = launch_site['Lat']
    lon = launch_site['Long']
    name = launch_site['Launch Site']

    folium.Circle(location=[lat,lon],radius=1000,color='pink',fill=True).add_child(folium.Popup(name)).add_to(site_map)
    folium.map.Marker(
        [lat,lon],icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name, )
    ).add_to(site_map)
site_map

In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [16]:
marker_cluster = MarkerCluster().add_to(site_map)


In [18]:
for index, launch in spacex_df.iterrows():
    lat = launch['Lat']
    lon = launch['Long']
    outcome = launch['class']
    launch_site = launch['Launch Site']

    # Determine the marker color based on the launch outcome
    if outcome == 1:
        marker_color = 'green'
    else:
        marker_color = 'red'

    # Create a marker and add it to the marker cluster
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=marker_color),
        popup=f"{launch_site}: {'Success' if outcome == 1 else 'Failure'}"
    ).add_to(marker_cluster)
site_map

In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [26]:

coastline_lat = 28.56367
coastline_lon = -80.57163

# Coordinates for the launch site (you can change these to the actual launch site coordinates)
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Calculate the distance between the launch site and the closest coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium.Marker on the selected closest coastline point on the map
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

# Add the distance marker to the map
distance_marker.add_to(site_map)

# Save the map to an HTML file
site_map.save("spacex_launch_records_with_distance.html")
site_map
